In [1]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split


In [2]:
# Settings:
pd.set_option('display.width', 190)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('max_colwidth', 200)
pd.options.display.float_format = '{:.4f}'.format
plt.style.use('default')
np.set_printoptions(threshold = 30, edgeitems = 30, precision = 2, suppress = False)


In [3]:
df_path = "../merged_data/features_USRECD.csv"
features = ["BCI", "BCIp", "BCIg", 'IE_SP_Comp', 'IE_SP_Dividend', 'IE_SP_Earnings', 'IE_Consumer_CPI', 'IE_Long_Interest', 'IE_Real_Price', 'IE_Real_Dividend', 'IE_Return_Price', 'IE_Real_Earnings',
                'IE_Scaled_Earnings', 'IE_Monthly_Returns', 'IE_Real_Returns', "YC_10_Year", "YC_3_Month", "YC_3_Month_Bond", "YC_Spread", "YC_Rec_Prob"]
model_names = ["Logistic Regression", "Penalized SVM", "Random Forest"]
get_models = [lambda: linear_model.LogisticRegression(), lambda: svm.SVC(kernel='linear', class_weight='balanced', probability=True), 
          lambda: RandomForestClassifier()]


In [4]:
# Read the data and do a little bit of wrangling:
df = pd.read_csv(df_path)
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date", drop=True)
df = df.drop(columns="Unnamed: 0")
df.head()

,BCI,BCIp,BCIg,USRECD,IE_SP_Comp,IE_SP_Dividend,IE_SP_Earnings,IE_Consumer_CPI,IE_Long_Interest,IE_Real_Price,IE_Real_Dividend,IE_Return_Price,IE_Real_Earnings,IE_Scaled_Earnings,IE_Monthly_Returns,IE_Real_Returns,YC_10_Year,YC_3_Month,YC_3_Month_Bond,YC_Spread,YC_Rec_Prob
Date,,,,,,,,,,,,,,,,,,,,,
1967-02-09,4.6052,6.5870,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153,1.5217,1.5518,1.5776,1.2065,-1.1432
1967-02-16,4.6052,6.5863,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153,1.5217,1.5518,1.5776,1.2065,-1.1432
1967-02-23,4.6012,6.5774,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153,1.5217,1.5518,1.5776,1.2065,-1.1432
1967-03-02,4.6032,6.5820,3.4751,0,4.4700,1.0613,1.7011,3.4935,1.5326,6.5860,3.1772,11.4869,3.8177,8.7185,0.0100,2.5153,1.5326,1.5173,1.5427,1.2692,-1.2586
1967-03-09,4.6042,6.5852,3.4751,0,4.4700,1.0613,1.7011,3.4935,1.5326,6.5860,3.1772,11.4869,3.8177,8.7185,0.0100,2.5153,1.5326,1.5173,1.5427,1.2692,-1.2586


In [5]:
# Split into training and test sets and hold out the test set until the end, so that it remains "unseen".
lag_of_y = 21 # This is the lag we introduce to the target variable so that we assess the indicator's 
              # ability to predict the target variable this many steps into the future.
              # With BCI, a lag of 21 data points corresponds to about half a year.

X_train, y_train = df.iloc[:-lag_of_y, df.columns != "USRECD"], df.iloc[lag_of_y:, df.columns == "USRECD"]

In [6]:
# for feature in features:
#     plt.figure()
#     X_train[feature].hist(bins = 50)
#     plt.xlabel(feature,fontsize=15)
#     plt.ylabel("Frequency",fontsize=15)
#     plt.show()

In [7]:
# Do a time series cross-validation on the test set by splitting it to k folds and doing a "rolling"
# validation against a validation fold, then averaging out the metrics.
splits = 4 # This is the number of splits/folds in the rolling validation.
tscv = TimeSeriesSplit(n_splits=splits)

for train_index, test_index in tscv.split(X_train): # Rolling cross-validation happens inside this loop.
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29 ... 529 530 531 532 533
 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551
 552 553 554 555 556 557 558] TEST: [ 559  560  561  562  563  564  565  566  567  568  569  570  571  572
  573  574  575  576  577  578  579  580  581  582  583  584  585  586
  587  588 ... 1084 1085 1086 1087 1088 1089 1090 1091 1092 1093 1094 1095
 1096 1097 1098 1099 1100 1101 1102 1103 1104 1105 1106 1107 1108 1109
 1110 1111 1112 1113]
TRAIN: [   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22   23   24   25   26   27
   28   29 ... 1084 1085 1086 1087 1088 1089 1090 1091 1092 1093 1094 1095
 1096 1097 1098 1099 1100 1101 1102 1103 1104 1105 1106 1107 1108 1109
 1110 1111 1112 1113] TEST: [1114 1115 1116 1117 1118 1119 1120 1121 1122 1123 1124 1125 1126 1127
 1128 1129 1130 1131 1132 1133 113

## Validation

In [8]:
AUC_ROCs = dict()
ACCs = dict()
for model_name, get_model in zip(model_names, get_models):
    print(model_name)
    AUC_ROCs[model_name] = 0
    ACCs[model_name] = 0
    for train_index, test_index in tscv.split(X_train): # Rolling cross-validation happens inside this loop.
        X_train_fold, X_validation_fold = X_train.iloc[train_index[:-lag_of_y], X_train.columns != "USRECD"], \
            X_train.iloc[test_index[:-lag_of_y], X_train.columns != "USRECD"]
        y_train_fold, y_validation_fold = y_train.iloc[train_index[lag_of_y:], y_train.columns == "USRECD"], \
            y_train.iloc[test_index[lag_of_y:], y_train.columns == "USRECD"]
            
        scalers = dict()
        for feature in features:
            scalers[feature] = StandardScaler()
            scalers[feature].fit(X_train_fold[[feature]])
            X_train_fold[feature] = scalers[feature].transform(X_train_fold[[feature]])
            X_validation_fold[feature] = scalers[feature].transform(X_validation_fold[[feature]])
            
        model = get_model()
        model.fit(X_train_fold[features], y_train_fold["USRECD"])
        positive_probs = [p[1] for p in model.predict_proba(X_validation_fold[features])]
        AUC_ROC = metrics.roc_auc_score(y_validation_fold, positive_probs)
        AUC_ROCs[model_name] += AUC_ROC
        predictions = model.predict(X_validation_fold[features])
        ACC = accuracy_score(y_validation_fold, predictions)
        ACCs[model_name] += ACC
        print(AUC_ROC, ACC)
        
    AUC_ROCs[model_name] /= splits
    ACCs[model_name] /= splits

Logistic Regression
0.9167965471765975 0.40074906367041196
0.862124248496994 0.9344569288389513
0.6414773085573751 0.8295880149812734
0.9867924528301887 0.7677902621722846
Penalized SVM
0.8052391799544419 0.25842696629213485
0.6703693100486687 0.9344569288389513
0.6364324301599648 0.799625468164794
0.965566037735849 0.7340823970037453
Random Forest
0.5407984654118211 0.7902621722846442
0.5732035499570569 0.7659176029962547
0.9051247556387026 0.8014981273408239
0.9570754716981131 0.3352059925093633


In [9]:
for model_name in model_names:
    print(model_name)
    print(f"AUC ROC: {AUC_ROCs[model_name]}")
    print(f"accuracy: {ACCs[model_name]}")

Logistic Regression
AUC ROC: 0.8517976392652888
accuracy: 0.7331460674157303
Penalized SVM
AUC ROC: 0.7694017394747311
accuracy: 0.6816479400749064
Random Forest
AUC ROC: 0.7440505606764234
accuracy: 0.6732209737827716
